# 1. Configurar dependencias y estilo de Plotly
Estas celdas iniciales establecen las librerías a utilizar, el estilo visual uniforme y los colores que destacarán a la Región de Los Ríos frente al resto de las regiones.

In [1]:
from pathlib import Path
import unicodedata

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

px.defaults.template = "plotly_white"
px.defaults.color_continuous_scale = "Tealgrn"

TARGET_REGION = "Region De Los Rios"
PALETTE = {
    "los_rios": "#E4572E",
    "otras": "#9CA3AF",
}

# Establecemos un color secundario para resaltar anotaciones y trazos secundarios.
SECONDARY_COLOR = "#1D4ED8"


# 2. Cargar y estandarizar dataset regional
Se carga el dataset procesado desde el ETL, se normalizan los nombres de región eliminando acentos y se asegura que las columnas financieras mantengan un formato numérico consistente.

In [2]:
from pathlib import Path

def normalize_region(name: str) -> str:
    """Elimina acentos y homogeniza capitalización de nombres de región."""

    if pd.isna(name):
        return name
    ascii_name = (
        unicodedata.normalize("NFKD", str(name))
        .encode("ascii", "ignore")
        .decode("utf-8")
    )
    return ascii_name.strip().title()


def coerce_numeric(series: pd.Series) -> pd.Series:
    """Convierte cualquier serie a float manteniendo nulos."""

    return pd.to_numeric(series, errors="coerce")


def resolve_path(relative: str) -> Path:
    notebook_root = Path.cwd()
    project_root = notebook_root.parent
    return (project_root / relative).resolve()


def load_dataset(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(
            f"No se encontró {path}. Ejecuta el ETL antes de continuar."
        )
    frame = pd.read_csv(path)
    frame.columns = [col.strip() for col in frame.columns]
    frame["Region_Normalizada"] = frame["Región"].map(normalize_region)

    currency_columns = [
        "Financiamiento Innova",
        "Aprobado Privado",
        "Aprobado Privado Pecuniario",
        "Monto Certificado Ley",
    ]
    for column in currency_columns:
        if column in frame:
            frame[column] = coerce_numeric(frame[column])

    frame["Año Adjudicación"] = pd.to_numeric(frame["Año Adjudicación"], errors="coerce")
    frame["anio_dt"] = pd.to_datetime(
        frame["Año Adjudicación"].astype("Int64"), format="%Y", errors="coerce"
    )
    frame["es_los_rios"] = frame["Region_Normalizada"].eq(TARGET_REGION)
    return frame


data_path = resolve_path("data/processed/corfo_projects.csv")
df = load_dataset(data_path)
df.head(2)

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final,Region_Normalizada,anio_dt,es_los_rios
0,25PATI-306231,Entorno para la innovación,Subsidio,Programa de Absorción Tecnológica para la Inno...,Programas de difusión y prospección tecnológica,VIGENTE,Persona Jurídica constituida en Chile,81494400-k,Corporación Universidad de Concepción,Incorporando inteligencia artificial mediante ...,...,NaN,NaN,NaN,NaN,NaN,NaN,Inteligencia Artificial (IA),Region Del Biobio,2025-01-01,False
1,25PATI-305719,Entorno para la innovación,Subsidio,Programa de Absorción Tecnológica para la Inno...,Programas de difusión y prospección tecnológica,VIGENTE,Persona Jurídica constituida en Chile,81494400-k,Universidad de Concepción,Realidad Extendida Industrial: Programa de Abs...,...,NaN,NaN,NaN,NaN,NaN,NaN,Realidad Virtual (VR) y/o Realidad Aumentada (AR),Region Del Biobio,2025-01-01,False


# 3. Preparar subconjuntos destacando la Región de Los Ríos
Creamos agregaciones de financiamiento y número de proyectos diferenciando a Los Ríos del resto para disponer de data frames listos para gráficas comparativas.

In [12]:
region_summary = (
    df.groupby("Region_Normalizada")
    .agg(
        total_innova=("Financiamiento Innova", "sum"),
        proyectos=("Código Proyecto", "count"),
        promedio_privado=("Aprobado Privado", "mean"),
    )
    .reset_index()
)
region_summary["es_los_rios"] = region_summary["Region_Normalizada"].eq(TARGET_REGION)
region_summary = region_summary.sort_values("total_innova", ascending=False)
region_summary["total_innova_mm"] = region_summary["total_innova"] / 1e6

# Seleccionamos las regiones con mayor financiamiento garantizando la presencia de Los Ríos
candidate_regions = region_summary["Region_Normalizada"].head(5).tolist()
if TARGET_REGION not in candidate_regions:
    candidate_regions.append(TARGET_REGION)
top_regions = list(dict.fromkeys(candidate_regions))[:5]

# Paleta específica por región para reutilizar en todas las visualizaciones
available_colors = iter(px.colors.qualitative.G10)
REGION_COLOR_MAP = {}
for region in top_regions:
    if region == TARGET_REGION:
        REGION_COLOR_MAP[region] = PALETTE["los_rios"]
    else:
        REGION_COLOR_MAP[region] = next(available_colors)

# Agregaciones temporales por región seleccionada
yearly_region_projects = (
    df[df["Region_Normalizada"].isin(top_regions)]
    .groupby(["anio_dt", "Region_Normalizada"])
    .size()
    .reset_index(name="proyectos")
)

panel_finance = (
    df[df["Region_Normalizada"].isin(top_regions)]
    .groupby(["anio_dt", "Region_Normalizada"])
    .agg(
        total_innova=("Financiamiento Innova", "sum"),
        aporte_privado=("Aprobado Privado", "sum"),
    )
    .reset_index()
)

region_summary.head()

,Region_Normalizada,total_innova,proyectos,promedio_privado,es_los_rios,total_innova_mm
3,Region Del Biobio,4.345746e+10,662,3.956706e+07,False,43457.463040
1,Region De Los Lagos,3.610510e+10,637,3.993489e+07,False,36105.098736
0,Region De La Araucania,1.862140e+10,380,2.472618e+07,False,18621.397189
2,Region De Los Rios,1.126368e+10,194,3.021250e+07,True,11263.679337


# 4. Visual comparativa de barras: Los Ríos vs otras regiones
Se grafica el financiamiento total aprobado por región, ordenado de mayor a menor, aplicando un color exclusivo para Los Ríos y anotaciones que resalten su posición relativa.

In [4]:
bar_df = region_summary.copy()
bar_df["color"] = np.where(bar_df["es_los_rios"], PALETTE["los_rios"], PALETTE["otras"])

fig_bar = go.Figure(
    go.Bar(
        x=bar_df["Region_Normalizada"],
        y=bar_df["total_innova_mm"],
        marker_color=bar_df["color"],
        text=bar_df["total_innova_mm"].round(1),
        textposition="outside",
        hovertemplate=(
            "Región: %{x}<br>Financiamiento: %{y:.1f} MM CLP<br>Proyectos: %{customdata}"
        ),
        customdata=bar_df["proyectos"],
    )
)

los_rios_rank = bar_df.index[bar_df["es_los_rios"].values][0]
los_rios_value = bar_df.loc[los_rios_rank, "total_innova_mm"]
los_rios_region = bar_df.loc[los_rios_rank, "Region_Normalizada"]

fig_bar.add_annotation(
    x=los_rios_region,
    y=los_rios_value,
    text="Región de Los Ríos",
    showarrow=True,
    arrowcolor=PALETTE["los_rios"],
    arrowhead=2,
    ay=-80,
)

fig_bar.update_layout(
    title="Financiamiento Innova por región (millones CLP)",
    xaxis_title="Región",
    yaxis_title="Financiamiento (MM CLP)",
    xaxis_tickangle=-35,
    bargap=0.25,
)

fig_bar.show()


# 5. Serie temporal: dinámica de proyectos con énfasis en Los Ríos
Se muestra la evolución anual del número de proyectos adjudicados comparando Los Ríos versus la suma del resto de las regiones, aplicando líneas con pesos distintos para enfatizar a Los Ríos.

In [8]:
line_fig = go.Figure()
for region in top_regions:
    region_data = yearly_region_projects[yearly_region_projects["Region_Normalizada"] == region]
    display_name = "Región de Los Ríos" if region == TARGET_REGION else region
    line_fig.add_trace(
        go.Scatter(
            x=region_data["anio_dt"],
            y=region_data["proyectos"],
            mode="lines+markers" if region == TARGET_REGION else "lines",
            name=display_name,
            line=dict(
                color=REGION_COLOR_MAP[region],
                width=4 if region == TARGET_REGION else 2,
                dash="solid" if region == TARGET_REGION else "dash",
            ),
            hovertemplate="Año %{x|%Y}<br>Proyectos %{y}<extra>" + display_name + "</extra>",
        )
    )

line_fig.update_layout(
    title="Conteo de proyectos adjudicados por año (Top regiones)",
    xaxis_title="Año",
    yaxis_title="Número de proyectos",
    legend_title="Región",
)

line_fig.show()

# 6. Panel interactivo y anotaciones para Los Ríos
Se combinan dos indicadores (financiamiento público y aporte privado) en un panel con subplots, controles de rango temporal y anotaciones que subrayan los hitos de Los Ríos frente al resto del país.

In [13]:
panel_finance = panel_finance.dropna(subset=["anio_dt"]).copy()


def build_metric_figure(metric_column: str, title: str, yaxis_title: str, annotate_peak: bool = False) -> go.Figure:
    fig = go.Figure()
    for region in top_regions:
        region_data = panel_finance[panel_finance["Region_Normalizada"] == region]
        display_name = "Región de Los Ríos" if region == TARGET_REGION else region
        fig.add_trace(
            go.Scatter(
                x=region_data["anio_dt"],
                y=region_data[metric_column] / 1e6,
                mode="lines+markers" if region == TARGET_REGION else "lines",
                name=display_name,
                line=dict(
                    color=REGION_COLOR_MAP[region],
                    width=4 if region == TARGET_REGION else 2,
                    dash="solid" if region == TARGET_REGION else "dash",
                ),
                hovertemplate="%{x|%Y}: %{y:.1f} MM<extra>" + title + " - " + display_name + "</extra>",
            )
        )

    if annotate_peak:
        los_rios_data = panel_finance[panel_finance["Region_Normalizada"] == TARGET_REGION]
        if not los_rios_data.empty:
            peak_row = los_rios_data.loc[los_rios_data[metric_column].idxmax()]
            fig.add_annotation(
                x=peak_row["anio_dt"],
                y=peak_row[metric_column] / 1e6,
                text=f"Pico Los Ríos: {peak_row['anio_dt'].year} ({peak_row[metric_column] / 1e6:.1f} MM)",
                arrowhead=2,
                arrowcolor=SECONDARY_COLOR,
                ax=60,
                ay=-60,
                showarrow=True,
            )

    fig.update_layout(
        title=title,
        xaxis_title="Año",
        yaxis_title=yaxis_title,
        hovermode="x unified",
        legend_title="Región",
        height=400,
    )

    fig.update_xaxes(
        rangeselector=dict(
            buttons=[
                dict(count=3, label="3y", step="year", stepmode="backward"),
                dict(count=5, label="5y", step="year", stepmode="backward"),
                dict(step="all", label="Todo"),
            ]
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
    return fig


fig_finance = build_metric_figure(
    metric_column="total_innova",
    title="Financiamiento Innova (MM CLP)",
    yaxis_title="Monto (MM CLP)",
    annotate_peak=True,
)

fig_finance.show()

In [14]:
fig_private = build_metric_figure(
    metric_column="aporte_privado",
    title="Aporte privado comprometido (MM CLP)",
    yaxis_title="Monto (MM CLP)",
)

fig_private.show()